In [1]:
from utils import *

In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
import requests

api_key = os.environ["GOOGLE_MAPS_API_KEY"]
place_name = "Cafe Ambrosia New York"

# Construct API request URL
url = f"https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={place_name}&inputtype=textquery&fields=place_id&key={api_key}"

# Send request
response = requests.get(url)
data = response.json()

# Extract Place ID
place_id = data.get("candidates", [{}])[0].get("place_id", "Not Found")

print(f"Place ID: {place_id}")


Place ID: ChIJ2z7wnbNZwokRG2yRG-3vIUs


In [ ]:
[
    {
        "url": "https://maps.app.goo.gl/DTZwWSq1UwBYaeodA?g_st=com.apple.shortcuts.Run-Workflow.(null)"
    },
    {
        "url": "https://maps.app.goo.gl/DTZwWSq1UwBYaeodA?g_st=com.apple.shortcuts.Run-Workflow.(null)"
    },
]


In [62]:
import PIL.Image
import PIL.ExifTags

# Open the image
img = PIL.Image.open("/Users/azamkhan/Downloads/Bakeri_2_STARS.jpg")

# Get EXIF data
exif_data = img._getexif()
if exif_data is None:
    raise ValueError("No EXIF metadata found.")

# Get the mapping for ExifTags
exif = {
    PIL.ExifTags.TAGS.get(tag): value
    for tag, value in exif_data.items()
    if tag in PIL.ExifTags.TAGS
}

# Extract GPS info
gps_info = exif.get("GPSInfo")

if gps_info is None:
    raise ValueError("No GPSInfo found in EXIF data.")

# Map GPS keys to tag names
gps_data = {}
for key in gps_info:
    decode = PIL.ExifTags.GPSTAGS.get(key, key)
    gps_data[decode] = gps_info[key]
print(gps_data)


AttributeError: module 'PIL' has no attribute 'Image'

In [6]:
gpsinfo = {}
for key in exif_data['GPSInfo'].keys():
    decode = ExifTags.GPSTAGS.get(key,key)
    gpsinfo[decode] = exif_data['GPSInfo'][key]
print (gpsinfo)

KeyError: 'GPSInfo'

In [10]:
latitude = gps_data.get('GPSLatitude', 0)
longitude = gps_data.get('GPSLongitude', 0)

print(latitude, longitude)

(40.0, 43.0, 14.87) (73.0, 59.0, 46.21)


In [11]:
def dms_to_decimal(dms, ref):
    """
    Convert GPS coordinates in DMS format to decimal degrees.

    Parameters:
        dms (tuple): (degrees, minutes, seconds) from Pillow GPSInfo
        ref (str): 'N', 'S', 'E', or 'W'

    Returns:
        float: decimal degrees
    """
    degrees, minutes, seconds = dms
    decimal = degrees + minutes / 60 + seconds / 3600

    # South and West are negative
    if ref in ["S", "W"]:
        decimal *= -1

    

    return decimal


In [73]:
def get_image_lat_long(image_path):
    """
    Get the latitude and longitude of an image using its EXIF data.

    Args:
        image_path (str): The path to the image file.

    Returns:
        tuple: (latitude, longitude) in decimal degrees
    """
    from PIL import Image, ExifTags
    
    img = Image.open(image_path)

    exif_data = img._getexif()
    if exif_data is None:
        raise ValueError("No EXIF metadata found.")

    exif = {
        ExifTags.TAGS[k]: v
        for k, v in exif_data.items()
        if k in ExifTags.TAGS
    }

    gpsinfo = {}

    if "GPSInfo" not in exif:
        raise ValueError("No GPSInfo found in EXIF data.")

    for key in exif["GPSInfo"].keys():
        decode = ExifTags.GPSTAGS.get(key, key)
        gpsinfo[decode] = exif["GPSInfo"][key]

    latitude = gpsinfo.get("GPSLatitude", 0)
    longitude = gpsinfo.get("GPSLongitude", 0)
    lat_ref = gpsinfo["GPSLatitudeRef"]  # get this from gpsinfo['GPSLatitudeRef']
    lon_ref = gpsinfo["GPSLongitudeRef"]  # get this from gpsinfo['GPSLongitudeRef']

    lat = dms_to_decimal(latitude, lat_ref)
    lon = dms_to_decimal(longitude, lon_ref)

    return (float(lat), float(lon))

In [78]:
get_image_lat_long("/Users/azamkhan/Downloads/Black Fox_5_STARS.jpg")

(40.761433333333336, -73.97358611111112)

In [17]:
# Use gps_data from Cell 4
lat_ref = gps_data.get('GPSLatitudeRef')
lon_ref = gps_data.get('GPSLongitudeRef')
latitude = gps_data.get('GPSLatitude')
longitude = gps_data.get('GPSLongitude')

if lat_ref and lon_ref and latitude and longitude:
    lat = dms_to_decimal(latitude, lat_ref)
    lon = dms_to_decimal(longitude, lon_ref)
    print(float(lat), float(lon))
else:
    print("Missing GPS reference or coordinate data")


40.720797222222224 -73.99616944444445


In [18]:
# map the lat long on a map
import folium

# Create a map centered on the latitude and longitude
map = folium.Map(location=[lat, lon], zoom_start=15)

# add a marker to the exact location
folium.Marker([lat, lon], popup="Librae").add_to(map)



In [20]:
import boto3

# AWS Credentials
AWS_ACCESS_KEY = os.environ["AWS_ACCESS_KEY_ID"]
AWS_SECRET_KEY = os.environ["AWS_SECRET_ACCESS_KEY"]
BUCKET_NAME = os.environ["BUCKET_NAME"]

# Initialize S3 Client
s3_client = boto3.client(
    "s3", aws_access_key_id=AWS_ACCESS_KEY, aws_secret_access_key=AWS_SECRET_KEY
)

In [21]:
# list all files in the bucket
s3_client.list_objects_v2(Bucket=BUCKET_NAME)



{'ResponseMetadata': {'RequestId': '99HH8E0XPY0W6C9Z',
  'HostId': 'HfFRqwlYk+1H70XcwM7xMdezeKZrcxAgKBXxlSIGeip+PCAEvo7FOufRfQGf7RHTxuhKZa5zQDg=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'HfFRqwlYk+1H70XcwM7xMdezeKZrcxAgKBXxlSIGeip+PCAEvo7FOufRfQGf7RHTxuhKZa5zQDg=',
   'x-amz-request-id': '99HH8E0XPY0W6C9Z',
   'date': 'Sat, 13 Dec 2025 20:13:30 GMT',
   'x-amz-bucket-region': 'us-east-1',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'IsTruncated': False,
 'Contents': [{'Key': 'Bakeri_2_STARS.jpg',
   'LastModified': datetime.datetime(2025, 2, 10, 22, 48, 52, tzinfo=tzutc()),
   'ETag': '"8af5112dc2afce13ce4fc516a5fbb5cd"',
   'ChecksumAlgorithm': ['CRC64NVME'],
   'Size': 2769405,
   'StorageClass': 'STANDARD'},
  {'Key': 'Bibliotheque_4_STARS.jpg',
   'LastModified': datetime.datetime(2025, 9, 1, 22, 28, 58, tzinfo=tzutc()),
   'ETag': '"b175496d48ff1c8b419fc9482b1fd0aa"',
   'ChecksumAlgor

In [29]:
import boto3

# Initialize the S3 client
s3_client = boto3.client('s3')

bucket_name = BUCKET_NAME
object_key = 'Variety_4_STARS.jpg'

try:
    # Retrieve the object metadata
    response = s3_client.head_object(Bucket=bucket_name, Key=object_key)

    # Print the entire response (contains all metadata)
    print("Full Metadata Response:")
    print(response)

    # Access specific system metadata
    print(f"\nContent Length: {response['ContentLength']} bytes")
    print(f"Content Type: {response['ContentType']}")
    print(f"Last Modified: {response['LastModified']}")

    # Access custom user-defined metadata
    if 'Metadata' in response:
        print("\nCustom User Metadata:")
        for key, value in response['Metadata'].items():
            print(f"- {key}: {value}")

except Exception as e:
    print(f"Error retrieving metadata: {e}")



Full Metadata Response:
{'ResponseMetadata': {'RequestId': 'K5XQ7S7T19XEH3GH', 'HostId': 'K7ft3fuN1g2+FfHOw0nIb+0Q2pXEOHPJ7+ccgFtG9MrNEsJn28F/4HB1+cCiQ06WPpZiITB0uUQiuH9B5msMFiRVB/8WMwA3F5kjXr/4Yhs=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'K7ft3fuN1g2+FfHOw0nIb+0Q2pXEOHPJ7+ccgFtG9MrNEsJn28F/4HB1+cCiQ06WPpZiITB0uUQiuH9B5msMFiRVB/8WMwA3F5kjXr/4Yhs=', 'x-amz-request-id': 'K5XQ7S7T19XEH3GH', 'date': 'Sat, 13 Dec 2025 20:17:16 GMT', 'last-modified': 'Mon, 01 Sep 2025 22:30:09 GMT', 'etag': '"55c3c8ff58c3d66a45f433b781d78b59"', 'x-amz-server-side-encryption': 'AES256', 'x-amz-meta-latitude': '40.74535', 'x-amz-meta-longitude': '-73.99465277777777', 'accept-ranges': 'bytes', 'content-type': 'binary/octet-stream', 'content-length': '1452377', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'AcceptRanges': 'bytes', 'LastModified': datetime.datetime(2025, 9, 1, 22, 30, 9, tzinfo=tzutc()), 'ContentLength': 1452377, 'ETag': '"55c3c8ff58c3d66a45f433b781d78b59"', 'ContentType': 'binary/oc

In [28]:
image = read_image_from_s3(BUCKET_NAME, "Variety_4_STARS.jpg")
lat, lon = get_image_lat_long(image)
(lat, lon)
# image_gps = {
#     "latitude": str(lat),
#     "longitude": str(lon),
# }
# write_image_to_s3(image, BUCKET_NAME, "Librae_5_STARS.jpeg", image_gps)


(40.74535, -73.99465277777777)

In [32]:
import boto3
from PIL import Image, ExifTags
from io import BytesIO


def read_image_from_s3(bucket, key, region_name="us-east-1"):
    """Load image file from s3 and return Pillow image object."""
    s3 = boto3.resource("s3", region_name=region_name)
    obj = s3.Bucket(bucket).Object(key)
    response = obj.get()
    file_stream = response["Body"]

    # Open as Pillow image
    img = Image.open(file_stream)

    return img


def get_image_lat_long(img):
    """
    Get the latitude and longitude of an image using its EXIF data.
    
    Args:
        img: Either a file path (str) or a PIL Image object
    """
    # If img is a string (file path), open it as an image
    if isinstance(img, str):
        img = Image.open(img)
    
    exif = {ExifTags.TAGS.get(k, k): v for k, v in (img._getexif() or {}).items()}

    gpsinfo = {}
    if "GPSInfo" not in exif:
        return None

    for key in exif["GPSInfo"].keys():
        decode = ExifTags.GPSTAGS.get(key, key)
        gpsinfo[decode] = exif["GPSInfo"][key]

    latitude = gpsinfo.get("GPSLatitude")
    longitude = gpsinfo.get("GPSLongitude")
    lat_ref = gpsinfo.get("GPSLatitudeRef")
    lon_ref = gpsinfo.get("GPSLongitudeRef")

    if not latitude or not longitude:
        return None

    lat = dms_to_decimal(latitude, lat_ref)
    lon = dms_to_decimal(longitude, lon_ref)

    return (float(lat), float(lon))


def dms_to_decimal(dms, ref):
    """
    Convert GPS coordinates in DMS format to decimal degrees.

    Parameters:
        dms (tuple): (degrees, minutes, seconds) from Pillow GPSInfo
        ref (str): 'N', 'S', 'E', or 'W'

    Returns:
        float: decimal degrees
    """
    degrees, minutes, seconds = dms
    decimal = degrees + minutes / 60 + seconds / 3600

    # South and West are negative
    if ref in ["S", "W"]:
        decimal *= -1

    return decimal


def write_image_to_s3(img, bucket, key, metadata, region_name="us-east-1"):
    """Write an image into S3 while preserving EXIF metadata."""
    s3 = boto3.resource("s3", region_name=region_name)
    obj = s3.Bucket(bucket).Object(key)

    file_stream = BytesIO()

    # Grab original EXIF if present
    exif_data = img.info.get("exif")

    if exif_data:
        img.save(file_stream, format="JPEG", exif=exif_data)
    else:
        img.save(file_stream, format="JPEG")

    file_stream.seek(0)

    obj.put(Body=file_stream.getvalue(), Metadata=metadata)


In [24]:
def full_pipeline(s3_key):
    try:
        image = read_image_from_s3(BUCKET_NAME, s3_key)
        (lat, lon) = get_image_lat_long(image)
        image_gps = {
            "latitude": str(lat),
            "longitude": str(lon),
        }
        write_image_to_s3(image, BUCKET_NAME, s3_key, image_gps)
    except Exception as e:
        print(f"Error: {e}")
        return None


In [25]:
full_pipeline("Bakeri_2_STARS.jpg")

Error: cannot unpack non-iterable NoneType object


In [119]:
s3_bucket_list = s3_client.list_objects_v2(Bucket=BUCKET_NAME)
keys = [obj['Key'] for obj in s3_bucket_list['Contents']]






In [110]:
keys

['Bakeri_2_STARS.jpg',
 'Bibliotheque_4_STARS.jpg',
 'Black Fox_5_STARS.jpg',
 'Cafe Grumpy_3_STARS.jpg',
 'Dear Coffee_5_STARS.jpg',
 'Devocion_5_STARS.jpg',
 'Dominique Ansel Bakery_4_STARS.jpg',
 'Fellini_5_STARS.jpg',
 'Frame_3_STARS.jpg',
 'Greenburg_5_STARS.jpg',
 'Hungarian Pastry Shop_3_STARS.jpg',
 'La Cabra_5_STARS.jpg',
 "Larry's Ca Phe_5_STARS.jpg",
 'Librae_5_STARS.jpeg',
 'L’Appartement 4F_5_STARS.jpg',
 'Nick and Sons_5_STARS.jpg',
 'Paloma_4_STARS.jpg',
 'RADIANT_4_STARS.jpg',
 'Rhythm Zero_4_STARS.jpg',
 'Solid State Coffee_5_STARS.jpg',
 'Sote Coffee Roasters_4_STARS.jpg',
 'Stone Street Cafe_4_STARS.jpeg',
 'Stone Street Café_4_STARS.jpg',
 'Super Nice Coffee and Bakery_4_STARS.jpg',
 'Tall Poppy_5_STARS.jpg',
 'Variety_4_STARS.jpg',
 'WatchHouse 5th Ave._5_STARS.jpg',
 'White Noise_5_STARS.jpg']

In [123]:
import boto3

s3client = boto3.client("s3")

cafe_locations = {}
paginator = s3client.get_paginator("list_objects_v2")
page_iterator = paginator.paginate(Bucket=BUCKET_NAME)
for bucket in page_iterator:
    for file in bucket["Contents"]:
        print(file["Key"])
        try:
            metadata = s3client.head_object(Bucket=BUCKET_NAME, Key=file["Key"])
            print(metadata['Metadata'])
            cafe_locations[file["Key"]] = {
                "latitude": metadata['Metadata']['latitude'],
                "longitude": metadata['Metadata']['longitude']
            }
        except:
            print("Failed {}".format(file["Key"]))


Bakeri_2_STARS.jpg
{}
Failed Bakeri_2_STARS.jpg
Bibliotheque_4_STARS.jpg
{'latitude': '40.72170555555556', 'longitude': '-74.00064166666667'}
Black Fox_5_STARS.jpg
{'latitude': '40.761433333333336', 'longitude': '-73.97358611111112'}
Cafe Grumpy_3_STARS.jpg
{'latitude': '40.728522222222225', 'longitude': '-73.94848611111111'}
Dear Coffee_5_STARS.jpg
{'latitude': '40.77822222222222', 'longitude': '-73.95665833333334'}
Devocion_5_STARS.jpg
{}
Failed Devocion_5_STARS.jpg
Dominique Ansel Bakery_4_STARS.jpg
{'latitude': '40.72518055555555', 'longitude': '-74.003075'}
Fellini_5_STARS.jpg
{'latitude': '40.7359', 'longitude': '-74.00145'}
Frame_3_STARS.jpg
{'latitude': '40.78015833333333', 'longitude': '-73.98046944444444'}
Greenburg_5_STARS.jpg
{'latitude': '40.727336111111114', 'longitude': '-73.95736666666667'}
Hungarian Pastry Shop_3_STARS.jpg
{'latitude': '40.80358611111111', 'longitude': '-73.96348611111111'}
La Cabra_5_STARS.jpg
{'latitude': '40.72419722222222', 'longitude': '-73.996063

In [125]:
cafe_locations

# map the lat long on a map
import folium

# Create a map centered on the latitude and longitude
map = folium.Map(location=[lat, lon], zoom_start=15)

# add cafe locations to the map 
for key, value in cafe_locations.items():
    folium.Marker([value['latitude'], value['longitude']], popup=key).add_to(map)

map
